# LLM based course finder

This is a LLM based model for finding a course in the University of Tartu. Given model is aimed for bachelor's students as it has only bachelor level courses. 

In [28]:
!pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.


In [154]:
#All imports
#BLOCK MUST BE RUNNED FOR THE PROTOTYPE TO WORK!!!!
import os
import openai
import numpy as np
import requests
import json
import math

In [134]:
#Keys and stuff
#BLOCK MUST BE RUNNED FOR THE PROTOTYPE TO WORK!!!!
openai.api_type = "azure"
openai.api_key = ""
openai.api_base = "https://tu-openai-api-management.azure-api.net/OLTATKULL"
openai.api_version = "2023-07-01-preview"

#Antoni arvuti path
#with open("/Users/tony/Desktop/GPT_key.txt") as f:
    #openai.api_key = f.readline()
    
#Rannari arvuti path
with open("/Users/rannarzirk/documents/ülikool/3.semester/GPT_kood.txt") as f:
    openai.api_key = f.readline().strip()

In [180]:
#Prework block
file = "uus_kõik_võtmed.txt"

#Checks whether a course with given code is a bachelor's course
def check_whether_bachelor(code):
    response = requests.get(f"https://ois2.ut.ee/api/courses/{code}/versions")
    data = response.json()
    print(code)
    if len(data) == 0:
        return False
    if "study_levels" in data[0]["additional_info"].keys():
        if data[0]["additional_info"]["study_levels"][0]["code"] == "bachelor":
            return True
        else:
            return False
    
#Retrieve all of the uuid keys from õis
def ask_api_for_keys(response, data):
    indeks = 1
    answer = []
    response = requests.get(f"https://ois2.ut.ee/api/courses?start={indeks}&take=300")
    data = response.json()
    while len(data) != 0:
        response = requests.get(f"https://ois2.ut.ee/api/courses?start={indeks}&take=300")
        data = response.json()
        print(indeks)
        for key in data:
            code = key["code"]
            if check_whether_bachelor(code):
                print("Add to list")
                answer.append(code)
        indeks += 300
    return answer

#Save all of the keys to a text file
def save_to_file(keys_list, file):
    with open(file, "w") as f:
        for key in keys_list:
            f.write(key +  "\n")


#keys = ask_api_for_keys(response, data)
#save_to_file(keys, file)

#Creates all the files with regular text
"""with open(file, "r") as f:
    for rida in f:
        course = requests.get(f"https://ois2.ut.ee/api/courses/{rida.strip()}")
        data = course.json()
        new_filename = ""
        description = ""
        for key in data:
            if key == "overview":
                for lower_key in data[key]:
                    if lower_key == "description":
                        description = description + data[key][lower_key]["et"] + "\n"
                    else:
                        for element in data[key][lower_key]:
                            if len(element) == 0 or "et" not in element.keys():
                                continue
                            else:
                                description = description + element["et"] + "\n"
            elif key == "code":
                new_filename = data[key]
            elif key == "title":
                new_filename = new_filename + " " + data[key]["et"] + ".txt"
                new_filename = new_filename.replace("/", " ")
        print(rida)
        with open("Bach_courses/" + new_filename, "w") as f2:
            f2.write(description)
            print("File created!")"""


'with open(file, "r") as f:\n    for rida in f:\n        course = requests.get(f"https://ois2.ut.ee/api/courses/{rida.strip()}")\n        data = course.json()\n        new_filename = ""\n        description = ""\n        for key in data:\n            if key == "overview":\n                for lower_key in data[key]:\n                    if lower_key == "description":\n                        description = description + data[key][lower_key]["et"] + "\n"\n                    else:\n                        for element in data[key][lower_key]:\n                            if len(element) == 0 or "et" not in element.keys():\n                                continue\n                            else:\n                                description = description + element["et"] + "\n"\n            elif key == "code":\n                new_filename = data[key]\n            elif key == "title":\n                new_filename = new_filename + " " + data[key]["et"] + ".txt"\n                new_filena

In [85]:
#BLOCK MUST BE RUNNED!!!!
#This block is for saving token info to a json file

#This function saves given data to a json file. For tokens
def save_to_json(file_name, amount1, amount2):
    data = None
    try:
        #Read info from json file
        with open(file_name) as f:
            data = json.load(f)
        #Increasing tokens
        data['usage']['prompt_tokens'] += amount1
        data['usage']['completion_tokens'] += amount2
        data['usage']['total_tokens'] =  data['usage']['prompt_tokens'] + data['usage']['completion_tokens']
        #Save data to json file
        with open("tokenCount.json", "w") as f:
            json.dump(data, f, indent=4)
    except Exception as e:
        print("An error occured (json): ", e)


In [177]:
#This block is for GPT generations
#BLOCK MUST BE RUNNED FOR THE PROTOTYPE TO WORK!!!!

#Generates a beautiful answer to the course request
def GPT_generate_answer(courses):
    #print(courses)
    try:
        # Making a request to the OpenAI API (ChatGPT as an example)
        response = openai.ChatCompletion.create(
            deployment_id = "IDS2023_ZIRK_GPT35",
            model="gpt-3.5-turbo",  # specify the model
            temperature=0.0,  # Setting the temperature
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": f"Ütle mulle, et need kursused on parimad valikud minu jaoks koos koodidega {' '.join(courses)}" }]
        )
        #print("Token count for output:", response['usage']['total_tokens'])

        #Calculating tokens
        prompt_tokens = response['usage']['prompt_tokens']
        completion_tokens = response['usage']['completion_tokens']
        # Print the response
        #print("Response from OpenAI:", response)
        save_to_json("tokenCount.json", prompt_tokens, completion_tokens)
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print("An error occurred:", e)
        
#GPT_generate_answer(['HVAJ.07.011 Keskaja õigusajalugu', 'OIAO.06.068 Õiguse süsteem ja meetod', 'P2OG.01.133 Andmekaitse', 'OIAO.01.034 Karistusõiguse eriseminar', 'KKSP.04.044 Sportliku treeningu seminar', 'FLAJ.03.140 Ajaloo ja ühiskonnaõpetuse didaktika'])       

def GPT_input_text(text):
    try:
        print(text)
        response = openai.Embedding.create(
            engine = "IDS2023_ZIRK_EMBEDDING",
            model="text-embedding-ada-002", # Use the3 appropriate embedding model
            input=text
        )
        input_tokens = response['usage']['prompt_tokens']
        # Print the response
        #print("Response from OpenAI:", response)
        #print("Token count for input:", input_tokens)
        #Saving the numbers of tokens into json file
        save_to_json("tokenCount.json", input_tokens, 0)
        return response
    except Exception as e:
        print("An error occurred:", e)

#GPT_input_text("Tahan õppida matemaatikat")
        
def GPT_input_file(filename):
    try:
        text = ""
        with open("Bach_courses/" + filename, "r") as f:
            text = f.read()
            #print(text)
        response = openai.Embedding.create(
            engine = "IDS2023_ZIRK_EMBEDDING",
            model="text-embedding-ada-002", # Use the3 appropriate embedding model
            input=text
        )
        # Print the response
        input_tokens = response['usage']['prompt_tokens']
    
        #print("Response from OpenAI:", response)
        #print("Token count for input:", input_tokens)
        #Saving the numbers of tokens into json file
        save_to_json("tokenCount.json", input_tokens, 0)
        return response
    except Exception as e:
        print("An error occurred:", e)
#GPT_input("FLAJ.05.136 Eesti arhitektuur ja kunst keskajal.txt")

In [175]:
#This block reads in all of the vectors from text files and calculatse the distances between vectors
#BLOCK MUST BE RUNNED FOR THE PROTOTYPE TO WORK!!!!
def read_in_all_vectors():
    all_vectors = {}
    #Works in Rannar's computer
    listdir = os.listdir("/Users/rannarzirk/Documents/Ülikool/3.semester/DataScience_project/Ained_kod")
    for vec in listdir:
        #A random file that appears in my computer. I don't know why it is there and I can't delete it.
        if vec == ".DS_Store":
            continue
        with open("Ained_kod/" + vec, "r",encoding="UTF-8") as f:
            embedding_vector = []
            for rida in f:
                embedding_vector.append(float(rida.strip()))
            all_vectors[vec] = embedding_vector
    return all_vectors

#Calculating the cosine similarity which shows whethe
def cosine_similarity(vec1, vec2):
    dot = sum(a*b for a, b in zip(vec1, vec2))
    length1 = math.sqrt(sum(a*a for a in vec1))
    length2 = math.sqrt(sum(b*b for b in vec2))
    cos_sim = dot / (length1 * length2)
    return cos_sim

#Sorting the arrays of 
def sort_arrays(names, values):
    for i in range(len(values)):
        j = i + 1
        for j in range(len(values)):
            if values[i] < values[j]:
                temp = values[i]
                values[i] = values[j]
                values[j] = temp
                temp_name = names[i]
                names[i] = names[j]
                names[j] = temp_name
    return names, values

#Finds the n closest vectors via cosine similarity
def calculate_distance_n(n, input_vec):
    i = 0
    closest_n_sims = [] #Distances
    closest_n_names = [] #Names of the courses (with codes)
    all_vectors = read_in_all_vectors()
    arr_sorted = False
    for vector in all_vectors.keys():
        dist = cosine_similarity(input_vec, all_vectors[vector])
        sim = 1 - dist
        if i > n:
            if not arr_sorted:
                arr_sorted = True
                arrays = sort_arrays(closest_n_names, closest_n_sims) #Returns array of lists first is the names and second is the values
                closest_n_sims = arrays[1]
                closest_n_names = arrays[0]
            for j in range(i):
                #Adding an course to the list and removing the last one
                if closest_n_sims[j] > sim:
                    closest_n_sims.insert(j, sim)
                    closest_n_names.insert(j, vector.replace(".txt", ""))
                    del closest_n_sims[len(closest_n_sims) - 1]
                    del closest_n_names[len(closest_n_names) - 1]
                    break
        else:
            #Just adding courses to the list
            closest_n_sims.append(sim)
            closest_n_names.append(vector.replace(".txt", ""))
            i += 1
    return closest_n_names

#print(calculate_distance_n(5, vec))

In [173]:
#This block saves all of the course descriptions as vectors to text files
#print(len(generated_gpt["data"][0]["embedding"]))

def save_vector_to_file(vector, filename):
    try:
        with open("Ained_kod/" + filename, "w") as f:
            for element in vector:
                f.write(str(element) + "\n")
            print("Vector written")
    except Exception as e:
        print("An error occured: ", e)
        
#Save all the vectors to files
def save_all_vectors(list_of_data):
    for i in range(len(list_of_data)):
        if list_of_data[i] == ".DS_Store":
            continue
        save_vector_to_file(GPT_input_file(list_of_data[i])["data"][0]["embedding"], list_of_data[i])

#Works in Rannar's computer        
#listdir = os.listdir("/Users/rannarzirk/Documents/Ülikool/3.semester/DataScience_project/Bach_courses")
#364
#save_all_vectors(listdir[729:])

In [179]:
#Prototype block
user_input = input("Sisesta kursuse otsing: ")
input_vec = GPT_input_text(user_input)["data"][0]["embedding"]
best_courses = calculate_distance_n(10, input_vec)
print(GPT_generate_answer(best_courses))

Sisesta kursuse otsing: Tahan õppida mängude loomise kohta
Tahan õppida mängude loomise kohta
Need kursused võivad olla head valikud sinu jaoks. Siin on koodid, mida sa mainisid:

- SHHI.02.033 Mäng koolieelses eas
- LTAT.02.018 Mängumootorid
- LOOM.01.123 Taimeanatoomia ja -morfoloogia
- HVVK.03.041 Laua- ja rollimängud
- P2VK.02.462 Põhipill II
- P2VK.02.461 Põhipill I
- SVNC.00.253 Karjääri planeerimine ja nõustamine
- SVHI.02.018 Elu mitmekesisus
- HVLC.09.029 Eesti keel algtasemele I (100% e-õpe), tase 0 > A1.1
- HVVK.03.027 Elamuspedagoogika
- KKSP.04.064 Kehalise kasvatuse algõpetus

Kui oled huvitatud nendest kursustest, soovitan sul uurida nende sisu ja eesmärke ning veenduda, et need vastavad sinu huvidele ja eesmärkidele. Samuti võid kaaluda, kuidas need kursused aitavad sul oma oskusi ja teadmisi arendada ning kas need on seotud sinu tulevaste karjääriplaanidega.
